In [1]:
import torch
from utils import DataLoader, evaluate_model
from models import SVRModel
from matplotlib import pyplot as plt

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
dl = DataLoader(
    benchmark="benchmark_1",
    split=None,
    embedding="bert_cls", 
    max_len=16,
    include_axes=True,
    shuffle_axes=False,  
    normalization="zscore"
)

INFO:utils.dataloader:Initializing DataLoader...
INFO:utils.dataloader:Using device: cuda
INFO:utils.dataloader:Loading dataset for benchmark benchmark_1 with split None
INFO:utils.dataloader:No split specified; loading all splits (train, validation, test).
INFO:utils.dataloader:Dataset loaded successfully.
INFO:utils.dataloader:Initializing embedding for method bert_cls...
INFO:utils.dataloader:Initialized BERT model and tokenizer.
INFO:utils.dataloader:Embedding resources initialized.


In [4]:
data = dl.preprocess()
X_train, y_train = data["train"]
X_valid, y_valid = data["validation"]
X_test, y_test = data["test"]

INFO:utils.dataloader:Starting preprocessing of dataset(s)...
INFO:utils.dataloader:Processing split: train with 878 samples...
INFO:utils.dataloader:Processing a single dataset split...
INFO:utils.dataloader:Combined feature vector shape: torch.Size([878, 6912])
INFO:utils.dataloader:Extracting and normalizing target values...
INFO:utils.dataloader:Normalizing target values...
INFO:utils.dataloader:Z-score parameters: mean=[ 5.44271868e+01  1.15348462e+01 -2.48804100e-02], std=[20.1219028   3.81394874  1.39502752]
INFO:utils.dataloader:Processing split: validation with 247 samples...
INFO:utils.dataloader:Processing a single dataset split...
INFO:utils.dataloader:Combined feature vector shape: torch.Size([247, 6912])
INFO:utils.dataloader:Extracting and normalizing target values...
INFO:utils.dataloader:Normalizing target values...
INFO:utils.dataloader:Z-score parameters: mean=[53.97165992 11.64925101  0.09862348], std=[21.3092882   3.78886061  1.38975187]
INFO:utils.dataloader:Proce

In [5]:
print("Shapes:", X_train.shape, y_train.shape)

if isinstance(X_train, torch.Tensor):
    X_train = X_train.cpu().numpy()
if isinstance(X_valid, torch.Tensor):
    X_valid = X_valid.cpu().numpy()
if isinstance(X_test, torch.Tensor):
    X_test = X_test.cpu().numpy()

Shapes: torch.Size([878, 6912]) (878, 3)


In [6]:
grid_params = {
    "estimator__C": [0.1, 1, 10],
    "estimator__epsilon": [0.01, 0.1, 1],
    "estimator__kernel": ['linear', 'rbf']
}
default_params = {
    "estimator__C": 1.0,
    "estimator__epsilon": 0.1,
    "estimator__kernel": "rbf"
}
svr_model = SVRModel(params=default_params)
svr_model.build_model()

INFO:models.svr:SVR model built (wrapped in MultiOutputRegressor).


In [7]:
best_model, best_params = svr_model.grid_search(X_train, y_train, X_valid, y_valid, param_grid=grid_params, cv=5)

print("Best Hyperparameters:", best_params)

INFO:models.svr:Starting grid search for SVR...


Fitting 5 folds for each of 18 candidates, totalling 90 fits


INFO:models.svr:Grid search complete.
INFO:models.svr:Best Hyperparameters: {'estimator__C': 1, 'estimator__epsilon': 1, 'estimator__kernel': 'rbf'}
INFO:models.svr:Validation MSE for best SVR model: 0.9066


Best Hyperparameters: {'estimator__C': 1, 'estimator__epsilon': 1, 'estimator__kernel': 'rbf'}


In [8]:
test_preds = best_model.model.predict(X_test)
mse, mae, r2, nll, crps = evaluate_model(y_test, test_preds)
print("Test Performance:")
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, NLL: {nll:.4f}, CRPS: {crps:.4f}")

Test Performance:
MSE: 0.9067, MAE: 0.7529, R2: 0.0933, NLL: 1.3700, CRPS: 0.7529


In [9]:
save_path = "baseline_weights/benchmark_1/svr.pkl"
best_model.save(save_path)

INFO:models.svr:SVR model saved to baseline_weights/benchmark_1/svr.pkl
